# Portfolio Project: Safety Option

**Names:** Deniz Makul, Richard Sha, Mitchel Shen

**Due:** 11/25/2022 at 11:59pm ET on Dropbox

**What to submit:** your .ipynb (python code in jupyter notebooks) file as well as a pdf with all the output from your code.  




In [158]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

In [159]:
#Getting tickers from csv file
csv_tickers = pd.read_csv('Tickers_Example.csv', header = None)

In [160]:
# Returns a list of of months that have less than 20 trading days

volume_start_date = '2022-01-01'
volume_end_date = '2022-10-31'

def remove_month(ticker):
    hist = ticker.history(start = volume_start_date, end = volume_end_date)
    
    months_to_remove = []
    for i in range(1,11,1):
        if i <= 9: #appends a 0 if single digit
            i = str(0) + str(i)
        df = hist.filter(like='2022-' + str(i), axis=0)
        if len(df.index) < 20:
            months_to_remove.append('2022-' + str(i))
    return months_to_remove

In [161]:
#Drops the tickers that are delisted or have a average monthly volume from 
# January 1st to October 31st of less that 200,000
volume_index_drop = []

# Sending in the first ticker to determine which month to remove

i = 0
while i < len(csv_tickers):
    stock_data = yf.Ticker(csv_tickers[0][i])
    stock_hist = stock_data.history(start = volume_start_date, end = volume_end_date, interval = '1mo')
    
    if stock_hist.empty:
        volume_index_drop.append(i)
        i+=1
        continue
    
    stock_volume = stock_hist['Volume']
    stock_volume = stock_volume.dropna()
    stock_volume = stock_volume.drop(remove_month(stock_data))
    
    mean = (stock_volume.mean())

    if mean<200000:
        volume_index_drop.append(i)         
    elif(stock_data.info['market'] != 'us_market'):
        volume_index_drop.append(i)
    i += 1
volume_index_drop

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


[4, 16, 40, 47, 48, 54, 56]

In [162]:
#Drops invalid tickers
csv_tickers.drop(index = volume_index_drop, inplace = True)
csv_tickers.reset_index(drop = True, inplace = True)
csv_tickers.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AIG


In [163]:
tickers_list = []
price_list = []
shares_list = []
values_list = []
weight_list = []

In [164]:
# Richard
total_amount_to_spend = 500000

Portfolio_Final = pd.DataFrame({
    'Ticker': tickers_list,
    'Price': price_list,
    'Shares': shares_list,
    'Value': values_list,
    'Weight': weight_list
})

In [165]:
# Deniz

In [166]:
# Mitchel
close_start = '2022-01-01'
close_end = '2022-10-31'
close_all = pd.DataFrame()
for i in range(0,len(csv_tickers)):
    stock_data = yf.Ticker(csv_tickers[0][i])
    stock_hist = stock_data.history(start = close_start, end = close_end, interval = '1d')
    stock_close = stock_hist['Close']
    stock_close = stock_close.dropna()
    close_all[csv_tickers[0][i]] = stock_close
    
close_all.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,180.959732,130.341812,136.745361,401.729431,56.390324,170.404495,166.056244,207.860001,45.393047,244.139999,...,31.361927,65.578415,153.253540,17.874918,227.143723,185.358856,497.457550,243.742798,206.660690,55.597622
2022-01-04,178.663071,130.091568,133.529343,398.858551,57.972282,167.522003,171.387100,213.630005,47.172203,241.729996,...,32.884541,65.578415,155.815720,18.022526,229.317703,185.553360,486.186798,247.933548,210.329269,57.053822
2022-01-05,173.910660,130.774933,132.929398,391.834412,57.176388,164.356995,169.560776,213.070007,46.376007,239.270004,...,32.884541,65.655388,153.951431,18.423183,224.117737,181.682800,484.988434,248.681198,207.686722,56.752872
2022-01-06,171.007492,130.158920,132.909729,372.912567,58.158970,163.253998,171.114563,211.339996,47.309818,237.300003,...,33.665623,65.578415,155.987808,18.352894,226.556168,181.624435,465.140808,249.369827,209.700104,58.296440
2022-01-07,171.176529,129.822067,133.322800,365.760132,59.681973,162.554001,172.579315,215.500000,48.341923,232.600006,...,34.634560,66.030556,154.180893,18.979887,225.997971,174.505737,454.196930,250.638855,211.171417,59.218700


In [167]:
close_all_pct = close_all.pct_change() * 100
close_all_pct.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,-1.269156,-0.191991,-2.351830,-0.714630,2.805373,-1.691559,3.210272,2.775909,3.919445,-0.987140,...,4.854974,0.000000,1.671857,0.825782,0.957095,0.104934,-2.265671,1.719333,1.775170,2.619176
2022-01-05,-2.659985,0.525295,-0.449298,-1.761060,-1.372888,-1.889309,-1.065614,-0.262134,-1.687850,-1.017661,...,0.000000,0.117375,-1.196470,2.223094,-2.267582,-2.085955,-0.246482,0.301553,-1.256386,-0.527483
2022-01-06,-1.669344,-0.471048,-0.014796,-4.829041,1.718510,-0.671098,0.916360,-0.811945,2.013565,-0.823338,...,2.375226,-0.117238,1.322740,-0.381528,1.088013,-0.032125,-4.092392,0.276912,0.969432,2.719805
2022-01-07,0.098848,-0.258801,0.310790,-1.917993,2.618689,-0.428778,0.856007,1.968394,2.181586,-1.980614,...,2.878120,0.689466,-1.158370,3.416318,-0.246384,-3.919461,-2.352810,0.508894,0.701627,1.582018


In [168]:
#Expected Return
expected_return = close_all_pct.mean()
expected_return.head()

AAPL   -0.049038
ABBV    0.071277
ABT    -0.138700
ACN    -0.140936
AIG     0.027678
dtype: float64

In [169]:
expected_return_from_0 = abs(0 - expected_return)
expected_return_from_0

expected_return_sorted = sorted(expected_return_from_0.items(), key=lambda x:x[1])

In [170]:
std_lst = close_all_pct.std()
std_lst = sorted(std_lst.items(), key=lambda x:x[1])

In [171]:
i = 0

correlation_pct_stocks = pd.DataFrame()
index = []

while(i < len(close_all_pct.columns)):
    j = 0
    
    correlation_temp = {}
    
    while(j < len(close_all_pct.columns)):
        correlation_df = pd.DataFrame({
            csv_tickers[0][i]: close_all_pct[csv_tickers[0][i]]
        })
        
        correlation_df[csv_tickers[0][j]] = close_all_pct[csv_tickers[0][j]]
        correlation = correlation_df.corr()[csv_tickers[0][i]][csv_tickers[0][j]]

        correlation_temp[csv_tickers[0][j]] = correlation
        
        j += 1
    
    correlation_pct_stocks[csv_tickers[0][i]] = list(correlation_temp.values())
    index.append(csv_tickers[0][i])
    i += 1


correlation_pct_stocks.set_index([pd.Index(index)])
correlation_pct_stocks.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
0,1.000000,0.206871,0.610150,0.788787,0.566180,0.662325,0.640718,0.576469,0.568125,0.298192,...,0.209095,0.376867,0.600640,0.384481,0.506570,0.730380,0.518132,0.477781,0.540758,0.468527
1,0.206871,1.000000,0.454250,0.312444,0.303831,0.199121,0.262372,0.110946,0.297603,0.231642,...,0.172512,0.381071,0.256924,0.268500,0.262419,0.235727,0.533026,0.247252,0.207049,0.378954
2,0.610150,0.454250,1.000000,0.683649,0.531133,0.486219,0.582679,0.430351,0.568027,0.331139,...,0.118772,0.446449,0.520299,0.373109,0.423279,0.528107,0.591592,0.493834,0.472782,0.549303
3,0.788787,0.312444,0.683649,1.000000,0.602921,0.679004,0.650519,0.548784,0.591080,0.385234,...,0.203884,0.413859,0.645052,0.364540,0.548193,0.737162,0.581465,0.560528,0.578152,0.526037
4,0.566180,0.303831,0.531133,0.602921,1.000000,0.459222,0.739157,0.540230,0.761078,0.243192,...,0.390822,0.289413,0.608583,0.373590,0.342745,0.493510,0.401822,0.515643,0.511382,0.735766


In [172]:
correlation_pct_mean = pd.DataFrame()
correlation_pct_mean = correlation_pct_stocks.sum()
correlation_pct_mean = correlation_pct_mean - 1
correlation_pct_mean = correlation_pct_mean/(len(correlation_pct_stocks) - 1)

correlation_pct_mean.head()

AAPL    0.498874
ABBV    0.290250
ABT     0.469198
ACN     0.525068
AIG     0.475104
dtype: float64

In [175]:
correlation_from_0 = abs(0 - correlation_pct_mean)
correlation_from_0 = sorted(correlation_from_0.items(), key=lambda x:x[1])

In [176]:
points_system = pd.DataFrame()
expected_lst = []
std_deviation_lst = []
correlation_lst = []

for i in expected_return_sorted:
    expected_lst.append(i[0])
    
for i in std_lst:
    std_deviation_lst.append(i[0])

for i in correlation_from_0:
    correlation_lst.append(i[0])

points_system['Expected Returns'] = expected_lst
points_system['Standard Deviation'] = std_deviation_lst
points_system['Correlation'] = correlation_lst
points_system.index += 1 

points_system = points_system.set_index([points_system.index[::-1]])

points_system.head()

,Expected Returns,Standard Deviation,Correlation
55,MON,MON,MON
54,SO,BMY,LMT
53,PM,MRK,OXY
52,AXP,KO,SLB
51,CVS,PEP,COP


In [177]:
score = pd.DataFrame()
    
score['Tickers'] = expected_lst

score['Points'] = None

for i in range(len(score)):
    score['Points'][i] = (int(points_system[points_system['Expected Returns'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Standard Deviation'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Correlation'] == expected_lst[i]].index.values)) 

score = score.sort_values("Points")
score = score[::-1]
score = score.reset_index(drop=True)
score.index += 1 

score.head()

,Tickers,Points
1,MON,165
2,SO,145
3,MO,142
4,PM,142
5,CL,135


In [178]:
number_of_stocks = 25

score_dropped = score.copy()
score_dropped.drop(labels = range(number_of_stocks + 1, len(score) + 1), inplace = True)
score_dropped.head()

,Tickers,Points
1,MON,165
2,SO,145
3,MO,142
4,PM,142
5,CL,135


In [179]:
score_dropped['Base Weighting'] = 100/(2*number_of_stocks)
weighting_sum = score_dropped['Points'].sum()

score_dropped['Points-based Weighting'] = (score_dropped['Points']/weighting_sum) * 50
score_dropped['Total Weighting'] = score_dropped['Base Weighting'] + score_dropped['Points-based Weighting']
score_dropped.head()

,Tickers,Points,Base Weighting,Points-based Weighting,Total Weighting
1,MON,165,2.0,2.850726,4.850726
2,SO,145,2.0,2.505183,4.505183
3,MO,142,2.0,2.453352,4.453352
4,PM,142,2.0,2.453352,4.453352
5,CL,135,2.0,2.332412,4.332412
